In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString                  as B
import qualified Data.ByteString.Char8            as B (lines)
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as AC

input <- B.lines <$> B.readFile "../input/day10.txt"

In [6]:
import Data.List

sortOn B.reverse $ filter (B.isInfixOf "value") input

["value 31 goes to bot 50","value 13 goes to bot 60","value 61 goes to bot 61","value 41 goes to bot 12","value 73 goes to bot 12","value 7 goes to bot 32","value 5 goes to bot 162","value 23 goes to bot 182","value 59 goes to bot 3","value 71 goes to bot 103","value 2 goes to bot 143","value 19 goes to bot 14","value 47 goes to bot 184","value 67 goes to bot 185","value 43 goes to bot 206","value 3 goes to bot 67","value 37 goes to bot 8","value 29 goes to bot 18","value 17 goes to bot 198","value 53 goes to bot 9","value 11 goes to bot 19"]

In [31]:
data Bot         = Bot Int                              deriving (Eq, Show, Ord)
data Sink        = OtherBot  Bot | Bucket Int           deriving (Eq, Show)
data Instruction = Input Int Bot | Output Bot Sink Sink deriving (Eq, Show)

data Storage = Holding (Maybe Int) (Maybe Int) deriving (Eq, Show)

parseInput = do
    value <- "value " *> AC.decimal <* " goes to bot "
    botN  <- AC.decimal
    return $ Input value (Bot botN)

parseSink = A.choice
    [ OtherBot . Bot <$> ("bot "    *> AC.decimal)
    , Bucket         <$> ("output " *> AC.decimal)
    ]

parseOutput = Output
    <$> (Bot <$> ("bot " *> AC.decimal))
    <*> (" gives low to " *> parseSink)
    <*> (" and high to "  *> parseSink)

parseInstruction = A.choice [parseInput, parseOutput]

parsed = map (either error id . A.parseOnly parseInstruction) input

head parsed

Output (Bot 135) (OtherBot (Bot 27)) (OtherBot (Bot 166))

In [38]:
import qualified Data.Map.Strict as M
import qualified Data.Sequence   as S

type Stored  = M.Map Bot Storage
type Outputs = M.Map Int [Int]
data State   = State Stored Outputs [Instruction]

full :: Bot -> Stored -> Bool
full bot stored = case M.lookup bot stored of
    Just (Holding (Just _) (Just _)) -> True
    _                                -> False

storeInput :: Bot -> Stored -> Int -> Stored
storeInput bot stored int = case M.lookup bot stored of
    Nothing                         -> M.insert bot (Holding (Just         int) Nothing)            stored
    Just (Holding (Just x) Nothing) -> M.insert bot (Holding (Just $ min x int) (Just $ max x int)) stored

putBucket :: Int -> Int -> Outputs -> Outputs
putBucket bucketNo chip outputs = case M.lookup bucketNo outputs of
    Nothing    -> M.insert bucketNo [chip]       outputs
    Just chips -> M.insert bucketNo (chip:chips) outputs

output :: Storage -> Sink -> Sink -> Stored -> Outputs -> (Stored, Outputs)
output (Holding (Just mn) (Just mx)) sink1 sink2 stored outputs = let
    (stored',outputs') = case sink1 of
        OtherBot bot -> (storeInput bot stored mn, outputs)
        Bucket b     -> (stored, putBucket b mn outputs)
    (stored'',outputs'') = case sink2 of
        OtherBot bot' -> (storeInput bot' stored' mx, outputs')
        Bucket b'     -> (stored', putBucket b' mx outputs')
    in (stored'',outputs'')

run (State stored outputs [])     = (stored,outputs)
run (State stored outputs (i:is)) = case i of
    Input int bot -> let
        stored' = storeInput bot stored int
        in run (State stored' outputs is)
    Output bot sink1 sink2
        | not $ full bot stored -> run (State stored outputs (is++[i]))
        | otherwise             -> let
            value              = stored M.! bot
            (stored',outputs') = output value sink1 sink2 stored outputs
            in run (State stored' outputs' is)
    
result = run (State M.empty M.empty parsed)

matcher (_,(Holding (Just 17) (Just 61))) = True
matcher _                                 = False

head $ filter matcher $ M.toList (fst result)

(Bot 93,Holding (Just 17) (Just 61))

In [48]:
outputs = snd result
head (outputs M.! 0) * head (outputs M.! 1) * head (outputs M.! 2)

47101